In [1]:
from datasets import load_dataset

# Load financial sentiment dataset
dataset = load_dataset('Jean-Baptiste/financial_news_sentiment_mixte_with_phrasebank_75')

# Use the columns 'summary_detail_with_title' for text and 'labels' for labels
dataset = dataset.rename_column("summary_detail_with_title", "text")
dataset = dataset.rename_column("labels", "label")

# Split the dataset into train and test
train_dataset = dataset['train']
test_dataset = dataset['test']


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from itertools import product

# Load pre-trained tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenize the dataset with padding to max length
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Define metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='macro')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Hyperparameter tuning
learning_rates = [2e-5, 3e-5]
num_epochs = [3, 4]
best_accuracy = 0
best_params = {}

for lr, epoch in product(learning_rates, num_epochs):
    training_args = TrainingArguments(
        output_dir='./results',  
        evaluation_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=epoch,
        weight_decay=0.01
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate()
    
    if eval_result['eval_accuracy'] > best_accuracy:
        best_accuracy = eval_result['eval_accuracy']
        best_params = {"learning_rate": lr, "num_train_epochs": epoch}

print(f"Best params: {best_params}, Best accuracy: {best_accuracy}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

{'eval_loss': 0.5947762727737427, 'eval_accuracy': 0.9222929936305733, 'eval_f1': 0.9076999897710191, 'eval_precision': 0.9185891654206259, 'eval_recall': 0.8988519580924644, 'eval_runtime': 51.7967, 'eval_samples_per_second': 15.155, 'eval_steps_per_second': 0.965, 'epoch': 3.0}
{'loss': 0.0109, 'grad_norm': 0.23762013018131256, 'learning_rate': 3.0215827338129496e-06, 'epoch': 3.6}


  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.5921365022659302, 'eval_accuracy': 0.9273885350318471, 'eval_f1': 0.9088065798592114, 'eval_precision': 0.9170096841329718, 'eval_recall': 0.9016239608644673, 'eval_runtime': 51.8497, 'eval_samples_per_second': 15.14, 'eval_steps_per_second': 0.964, 'epoch': 4.0}
{'train_runtime': 3877.8262, 'train_samples_per_second': 4.586, 'train_steps_per_second': 0.287, 'train_loss': 0.020876711381853912, 'epoch': 4.0}


  0%|          | 0/50 [00:00<?, ?it/s]

Best params: {'learning_rate': 3e-05, 'num_train_epochs': 4}, Best accuracy: 0.9273885350318471


In [5]:
# Retrain with best params
training_args = TrainingArguments(
    output_dir='./results',  # This is required by TrainingArguments
    evaluation_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=best_params["num_train_epochs"],
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

c:\Users\Mulla\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/1112 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.7002525329589844, 'eval_accuracy': 0.9197452229299363, 'eval_f1': 0.8993005539491791, 'eval_precision': 0.8976980555568401, 'eval_recall': 0.9009923652961628, 'eval_runtime': 51.0385, 'eval_samples_per_second': 15.381, 'eval_steps_per_second': 0.98, 'epoch': 1.0}
{'loss': 0.0273, 'grad_norm': 0.020316896960139275, 'learning_rate': 1.6510791366906477e-05, 'epoch': 1.8}


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.6593212485313416, 'eval_accuracy': 0.9286624203821656, 'eval_f1': 0.9070949405446073, 'eval_precision': 0.9251331666008512, 'eval_recall': 0.8913217404356645, 'eval_runtime': 52.2248, 'eval_samples_per_second': 15.031, 'eval_steps_per_second': 0.957, 'epoch': 3.0}
{'loss': 0.0122, 'grad_norm': 0.0023053139448165894, 'learning_rate': 3.0215827338129496e-06, 'epoch': 3.6}


  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.6130384802818298, 'eval_accuracy': 0.9248407643312102, 'eval_f1': 0.9023609708662198, 'eval_precision': 0.9118815925538614, 'eval_recall': 0.8935576277348428, 'eval_runtime': 52.2878, 'eval_samples_per_second': 15.013, 'eval_steps_per_second': 0.956, 'epoch': 4.0}
{'train_runtime': 3958.6091, 'train_samples_per_second': 4.492, 'train_steps_per_second': 0.281, 'train_loss': 0.01782830698053828, 'epoch': 4.0}


TrainOutput(global_step=1112, training_loss=0.01782830698053828, metrics={'train_runtime': 3958.6091, 'train_samples_per_second': 4.492, 'train_steps_per_second': 0.281, 'total_flos': 588960557512704.0, 'train_loss': 0.01782830698053828, 'epoch': 4.0})

In [6]:
# Save the best model
model.save_pretrained('./group2_fin_model')
tokenizer.save_pretrained('./group2_fin_model')

('./group2_fin_model\\tokenizer_config.json',
 './group2_fin_model\\special_tokens_map.json',
 './group2_fin_model\\vocab.txt',
 './group2_fin_model\\added_tokens.json',
 './group2_fin_model\\tokenizer.json')